In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.style.use('classic')
%matplotlib inline
import requests

In [2]:
from sklearn.feature_selection import SelectKBest,SelectPercentile,f_classif
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
cd '/content/drive/MyDrive/03_Validation'

/content/drive/MyDrive/03_Validation


In [ ]:
!dir

abalone_app.csv   abalone_dataset.csv	README.md
abalone_app.xlsx  abalone_dataset.xlsx


In [ ]:
data = pd.read_csv('abalone_dataset.csv')
data.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,type
0,M,0.535,0.420,0.150,0.6995,0.2575,0.1530,0.2400,3
1,I,0.510,0.380,0.115,0.5155,0.2150,0.1135,0.1660,1
2,I,0.185,0.130,0.045,0.0290,0.0120,0.0075,0.0095,1
3,M,0.550,0.450,0.170,0.8100,0.3170,0.1570,0.2200,3
4,I,0.535,0.415,0.150,0.5765,0.3595,0.1350,0.2250,1


In [ ]:
data.type.value_counts()

1    1078
3    1051
2    1003
Name: type, dtype: int64

In [ ]:
data_test = pd.read_csv('abalone_app.csv')
data_test.head(5)

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight
0,M,0.600,0.480,0.175,1.2290,0.4125,0.2735,0.4150
1,F,0.545,0.385,0.150,1.1185,0.5425,0.2445,0.2845
2,F,0.645,0.520,0.180,1.2850,0.5775,0.3520,0.3170
3,M,0.640,0.510,0.170,1.3715,0.5670,0.3070,0.4090
4,F,0.655,0.540,0.215,1.5555,0.6950,0.2960,0.4440


#Data preprocessing


In [ ]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
new_sex_coders = encoder.fit_transform(data.sex.copy())

df_sex = pd.DataFrame(new_sex_coders,columns=['sex0','sex1','sex2'])
df_sex.head(5)

#data = data.drop('sex',axis=1)
new_data = pd.concat([df_sex.reset_index(drop=True), data.reset_index(drop=True)], axis=1)
new_data = new_data.drop('sex',axis=1)
new_data.head(5)

,sex0,sex1,sex2,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,type
0,0,0,1,0.535,0.420,0.150,0.6995,0.2575,0.1530,0.2400,3
1,0,1,0,0.510,0.380,0.115,0.5155,0.2150,0.1135,0.1660,1
2,0,1,0,0.185,0.130,0.045,0.0290,0.0120,0.0075,0.0095,1
3,0,0,1,0.550,0.450,0.170,0.8100,0.3170,0.1570,0.2200,3
4,0,1,0,0.535,0.415,0.150,0.5765,0.3595,0.1350,0.2250,1


Index(['length', 'diameter', 'height', 'whole_weight', 'shucked_weight',
       'viscera_weight', 'shell_weight', 'type', 'sex_0', 'sex_1', 'sex_2'],
      dtype='object')

#Data split

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(new_data.drop('type',axis=1),new_data.type,test_size=0.2,random_state=42)
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(2505, 10) (627, 10)
(2505,) (627,)


#Hyperparameters Selection

In [ ]:
def get_neighbors(n,d):
  neighbor = []
  neighbor.append([n+2,d+2])
  neighbor.append([n-2,1 if d-2 < 1 else d-2])
  neighbor.append([n+1,d+1])
  neighbor.append([n-1,1 if d-1 < 1 else d-1])
  neighbor.append([np.random.randint(10,100),np.random.randint(1,5)])

  return neighbor

def hill_climbing(n,d,iter):

  best_score = 0
  best_n = n
  best_d = d

  for i in range(iter):
    for _n in get_neighbors(n,d):

      rfc = RandomForestClassifier(n_estimators=_n[0],max_depth=_n[1])
      rfc.fit(x_train,y_train)
      current_score = rfc.score(x_test,y_test)

      #print('n={},d={} -> score={}'.format(_n[0],_n[1],current_score))
      if best_score < current_score:
        best_score = current_score
        best_n = _n[0]
        best_d = _n[1]

    n = best_n
    d = best_d

    print('i = {}, n={},d={} -> score={}'.format(i,best_n, best_d,best_score))

  return best_n,best_d,best_score

In [ ]:
_n = np.random.randint(0,10)
_d = np.random.randint(1,10)

n,d,score = hill_climbing(_n,_d,50)

i = 0, n=7,d=2 -> score=0.5964912280701754
i = 1, n=8,d=3 -> score=0.6331738437001595
i = 2, n=10,d=5 -> score=0.6475279106858054
i = 3, n=10,d=5 -> score=0.6475279106858054
i = 4, n=10,d=5 -> score=0.6475279106858054
i = 5, n=10,d=5 -> score=0.6475279106858054
i = 6, n=12,d=7 -> score=0.6539074960127592
i = 7, n=12,d=7 -> score=0.6539074960127592
i = 8, n=12,d=7 -> score=0.6539074960127592
i = 9, n=12,d=7 -> score=0.6539074960127592
i = 10, n=12,d=7 -> score=0.6539074960127592
i = 11, n=12,d=7 -> score=0.6539074960127592
i = 12, n=12,d=7 -> score=0.6539074960127592
i = 13, n=12,d=7 -> score=0.6539074960127592
i = 14, n=12,d=7 -> score=0.6539074960127592
i = 15, n=12,d=7 -> score=0.6539074960127592
i = 16, n=12,d=7 -> score=0.6539074960127592
i = 17, n=12,d=7 -> score=0.6539074960127592
i = 18, n=12,d=7 -> score=0.6539074960127592
i = 19, n=12,d=7 -> score=0.6539074960127592
i = 20, n=12,d=7 -> score=0.6539074960127592
i = 21, n=12,d=7 -> score=0.6539074960127592
i = 22, n=12,d=7 -> sc

In [ ]:
print('Best_n = {},Best_d = {}, Best_score= {}'.format(n,d,score))

Best_n = 13,Best_d = 8, Best_score= 0.6555023923444976


#Define Model

In [ ]:
from sklearn.model_selection import cross_val_score
rfc = RandomForestClassifier(n_estimators=13,max_depth=8)
rfc.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=13,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#Model Valitation

In [ ]:
rfc.score(x_test,y_test)

0.6363636363636364

#Model Predict

In [ ]:
data_test = pd.read_csv('abalone_app.csv')
data_test.head(5)

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight
0,M,0.600,0.480,0.175,1.2290,0.4125,0.2735,0.4150
1,F,0.545,0.385,0.150,1.1185,0.5425,0.2445,0.2845
2,F,0.645,0.520,0.180,1.2850,0.5775,0.3520,0.3170
3,M,0.640,0.510,0.170,1.3715,0.5670,0.3070,0.4090
4,F,0.655,0.540,0.215,1.5555,0.6950,0.2960,0.4440


In [ ]:
test_sex_coders = encoder.transform(data_test.sex.copy())

df_sex_test = pd.DataFrame(test_sex_coders,columns=['sex0','sex1','sex2'])
df_sex_test.head(5)

#data = data.drop('sex',axis=1)
new_data_test = pd.concat([df_sex_test.reset_index(drop=True), data_test.reset_index(drop=True)], axis=1)
new_data_test = new_data_test.drop('sex',axis=1)
new_data_test.head(5)

,sex0,sex1,sex2,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight
0,0,0,1,0.600,0.480,0.175,1.2290,0.4125,0.2735,0.4150
1,1,0,0,0.545,0.385,0.150,1.1185,0.5425,0.2445,0.2845
2,1,0,0,0.645,0.520,0.180,1.2850,0.5775,0.3520,0.3170
3,0,0,1,0.640,0.510,0.170,1.3715,0.5670,0.3070,0.4090
4,1,0,0,0.655,0.540,0.215,1.5555,0.6950,0.2960,0.4440


In [ ]:
y_pred = rfc.predict(new_data_test)
y_pred

array([3, 2, 2, ..., 3, 1, 2])

#Submit predicted values

In [139]:
# Enviando previsões realizadas com o modelo para o servidor
URL = 'https://aydanomachado.com/mlclass/03_Ranking.php'

#TODO Substituir pela sua chave aqui
DEV_KEY = "Andre Santos"

# json para ser enviado para o servidor
data = {'dev_key':DEV_KEY,
        'predictions':pd.Series(y_pred).to_json(orient='values')}

# Enviando requisição e salvando o objeto resposta
r = requests.post(url = URL, data = data)

# Extraindo e imprimindo o texto da resposta
pastebin_url = r.text
print(" - Resposta do servidor:\n", r.text, "\n")

 - Resposta do servidor:
 
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 80%;
    margin: 0 auto;
}

td, th {
    border: 1px solid #ddd;
    text-align: left;
    padding: 8px;
}

td.prize {
    text-align: center;
}

tr:nth-child(even) {
    background-color: #f2f2f2;
}

tr:hover {
	background-color: #ddd;
}

th {
    padding-top: 12px;
    padding-bottom: 12px;
    text-align: center;
    background-color: #4CAF50;
    color: white;
}

h1 {
	text-align: center;
}

</style>
</head>
<body style="width:80%; margin:auto;">

<h1>Top 10</h1>
<h1>Atividade 03 - Avaliação de classificador para Abalones</h1>

<table>
  <tr>
    <th>Equipe</th>
    <th>Acurácia</th>
    <th>Data do envio</th>
    <th>Rank</th>
  </tr>

<tr>
    	<td>Simple Agent*</td>
        <td>0.5980861244</td>
        <td>11-12-2020 11:28:24</td>
        <td class="prize"><img 